In [1]:
import sympy as sp
import numpy as np
from pint import UnitRegistry
ureg = UnitRegistry()
AssignQuantity = ureg.Quantity

In [2]:
M = AssignQuantity(18.0153,'g/mol')
R = AssignQuantity(8.314,'J/mol/K')
n = AssignQuantity(1,'mol')

### Defining heat capacity functions for water

In [3]:
def CP_H2Ogas(T,AssignQuantity):
    """ www.engineeringtoolbox.com/water-vapor-d_979.html """
    m = AssignQuantity(0.0067,'J/mol/K^2')
    CP0 = AssignQuantity(33.58,'J/mol/K')
    T0 = AssignQuantity(300,'K')
    CP = CP0 + m*(T-T0)
    return CP

In [4]:
def CP_H2Oice(T,AssignQuantity):
    """ www.liquisearch.com/heat_capacity/table_of_specific_heat_capacities """
    CP = AssignQuantity(38.0,'J/mol/K')
    return CP

### Your turn
In the cell below, create a similar function for evaluating the heat capacity of liquid water. Call your function "CP_H2Oliq".

In [5]:
### BEGIN SOLUTION ###
def CP_H2Oliq(T,AssignQuantity):
    """ webbook.nist.gov """
    A = AssignQuantity(-203.606,'J/mol/K')
    B = AssignQuantity(1523.290,'J/mol/K^2')
    C = AssignQuantity(-3196.413,'J/mol/K^3')
    D = AssignQuantity(2474.455,'J/mol/K^4')
    E = AssignQuantity(3.855326,'J/mol K')
    t = T/1000
    CP = A + B*t + C*t**2 + D*t**3 + E/t**2
    return CP
### END SOLUTION ###

### Examining the heat capacities
The cell below shows how to use these functions for a temperature of 298 K.

In [6]:
# Examining values at 298 K
T = AssignQuantity(298,'K')
CP_gas = CP_H2Ogas(T,AssignQuantity); print(CP_gas)
CP_liq = CP_H2Oliq(T,AssignQuantity); print(CP_liq)
CP_ice = CP_H2Oice(T,AssignQuantity); print(CP_ice)

33.5666 joule / kelvin / mole
75.37700524361435 joule / kelvin / mole
38.0 joule / kelvin / mole


### Your turn
In the cell below, evaluate the heat capacities at 373 K.

In [7]:
# Examining values at 373 K
### BEGIN SOLUTION
T = AssignQuantity(373,'K')
CP_gas = CP_H2Ogas(T,AssignQuantity); print(CP_gas)
CP_liq = CP_H2Oliq(T,AssignQuantity); print(CP_liq)
CP_ice = CP_H2Oice(T,AssignQuantity); print(CP_ice)
### END SOLUTION

34.0691 joule / kelvin / mole
75.98999884816186 joule / kelvin / mole
38.0 joule / kelvin / mole


### Comparison to values predicted from equipartition
Below, print the values of $C_P$ you'd expect for water if it were an ideal gas, in the low-temperature (non-vibrating) and classical (vibrating) limits.

In [8]:
### BEGIN SOLUTION ###
CP_equipartition_LowTemp = (3/2 + 3/2 + 1)*R*n
CP_equipartition_Classical = (3/2 + 3/2 + 3 + 1)*R*n

print(CP_equipartition_LowTemp)
print(CP_equipartition_Classical)
### END SOLUTION ### 

33.256 joule / kelvin
58.198 joule / kelvin


### Pause for analysis
From your results, would you say that water vapor at 298 K is vibrating? (Explain a little.)

### Comparison to other units
Below, do these for comparison:
- Create a new variable for the liquid (say, CP_liq_mass) that represents the heat capacity of liquid water on a per-gram basis, and print the results.
- Convert your result to cal/g/K (you can use the .ito converter for this if you like!). You should get a number close to 1.

In [9]:
### BEGIN SOLUTION ###
CP_liq_mass = CP_liq/M; print(CP_liq_mass)

CP_liq_mass.ito('cal/g/K')
print(CP_liq_mass)
### END SOLUTION ###

4.218081233627076 joule / gram / kelvin
1.008145610331519 calorie / gram / kelvin


### Symbolic variables with units(!)
Turns out that you can assign symbolic variables units! Which will be very useful, because then we can use our existing functions CP_H2Oliq and CP_H2Ogas. Sadly, however, not all sympy functions work properly with symbolic variables that have units, so in the end we'll have to de-dimensionalize our result.

We do all this in three steps below -- you should just fill in the missing parts of Step 2, with a line like

    CP_liq = CP_H2Oliq(T,AssignQuantity)
    
for the liquid, and another one like it for the gas.

In [10]:
# Step 1. Get a symbolic temperature with units
T = AssignQuantity(sp.var("T"),'K')

# Step 2. Get the heat capacity of the liquid and gas using symbolic T
### BEGIN SOLUTION ###
CP_liq = CP_H2Oliq(T,AssignQuantity) ; print(CP_liq)
CP_gas = CP_H2Ogas(T,AssignQuantity) ; print(CP_gas)
### END SOLUTION

# Step 3. Strip the dimensions out of CP_liq
T = sp.var("T")
CP_liq = CP_liq.subs(T,T); print(CP_liq)
CP_gas = CP_gas.subs(T,T); print(CP_gas)

2.474455e-6*T**3 - 0.003196413*T**2 + 1.52329*T - 203.606 + 3855326.0/T**2 joule / kelvin / mole
0.0067*T + 31.57 joule / kelvin / mole
2.474455e-6*T**3 - 0.003196413*T**2 + 1.52329*T - 203.606 + 3855326.0/T**2
0.0067*T + 31.57


### Getting $\Delta(\Delta H_{vap})$
We can find the *change* in $\Delta H_{vap}$ in going from 298 to 373 K   

$$
\Delta(\Delta H_{vap}) = \int_{298}^{373} \Delta C_P \ dT
$$

where $\Delta C_P = C_P(gas) - C_P(liq)$. From that, we can find the enthalpy of vaporization at 373 K using

$$
\Delta H_{vap}(373 \ K) = \Delta H_{vap}(298 \ K) + \Delta(\Delta H_{vap}) 
$$

where $\Delta H_{vap}(298 \ K) = 44,000$ J/mol.

That's your challenge in the cell below. Hopefully, you'll end up with a value close to $41,000$ J/mol.

In [11]:
### BEGIN SOLUTION ###
DeltaCP = CP_gas - CP_liq; print(DeltaCP)
DeltaDeltaHvap = sp.integrate(DeltaCP,T).subs(T,373) - sp.integrate(DeltaCP,T).subs(T,298)
print(DeltaDeltaHvap)
DeltaHvap = 44000 + DeltaDeltaHvap
print(DeltaHvap)
### END SOLUTION ###

-2.474455e-6*T**3 + 0.003196413*T**2 - 1.51659*T + 235.176 - 3855326.0/T**2
-3123.73463662321
40876.2653633768
